In [80]:
import dhlab as dh
import pandas as pd
import dhlab.graph_networkx_louvain as gnl
import networkx as nx
from IPython.display import HTML

In [5]:
import igraph as ig
import leidenalg as la

from igraph import Graph
from leidenalg import find_partition
import numpy as np
import os

In [7]:
import conc_coll_corpus as co
import polnet_graphs as polg

In [8]:
from importlib import reload

In [12]:
zotero = co.zot()

In [32]:
G = polg.make_nx_graph(zotero)

## convert to igraph and use leiden algorithm

In [91]:
reload(polg)

<module 'polnet_graphs' from '/mnt/disk1/Github/polnet_plotly/polnet_graphs.py'>

In [92]:
h = polg.igraph_from_networkx(G)

In [90]:
polg.degree(h)

,vertex_id,label,degree
177,A6HRG4JG,NOU 2019: 23,73
211,FQTXYHMF,NOU 2018: 15,57
4,3ETZFFNS,NOU 1995: 18,47
135,V4RID8TN,NOU 2009: 18,46
35,JQNL2KII,NOU 2000: 14,46
...,...,...,...
453,UMG9ZHNR,NOU 1975: 7,1
454,7UEWRDBF,NOU 1982: 33,1
455,298BNUC8,St.meld. nr. 88 (1981-82),1
456,9FY2LAJF,NOU 1984: 5,1


In [94]:
import json

# Assuming 'h' is your igraph graph

# Extract nodes
nodes = [{"id": vertex['name'], "label": vertex['label'], 'centrality':vertex['centrality']} for vertex in h.vs]

# Extract edges
links = [{"source": edge.source_vertex['name'], "target": edge.target_vertex['name']} for edge in h.es]

# Create a graph dictionary
graph = {"nodes": nodes, "links": links}

# Convert to JSON (if needed)
graph_json = json.dumps(graph)

# 'graph_json' is now ready to be used with D3.js


In [96]:
with open('test.html', "w", encoding = "utf-8") as fp:
    fp.write(f"""
<!DOCTYPE html>
<html lang="en">
   <head>
      <meta charset="UTF-8">
      <title>D3 Force Layout</title>
      <style>
         text {{
         font-family: sans-serif;
         }}
      </style>
   </head>
   <body>
      <svg width="1800" height="1600"></svg>
      <script src="https://d3js.org/d3.v5.min.js"></script>
      <script>
         // Load the network graph
          var graph = JSON.parse('{graph_json}'); 

         
         d3.select("svg").selectAll("*").remove();
     
         var nodes = graph.nodes;
         var links = graph.links;

         var svg = d3.select("svg"),
             width = +svg.attr("width"),
             height = +svg.attr("height");

        var centralityScale = d3.scaleLinear()
         .domain([d3.min(nodes, d => d.centrality), d3.max(nodes, d => d.centrality)])
         .range([5, 35]);  // Min and max circle radii
         
        var fontSizeScale = d3.scaleLinear()
        .domain([d3.min(nodes, d => d.centrality), d3.max(nodes, d => d.centrality)])
        .range([10, 50]);  // Min and max font sizes, adjust as needed

         var simulation = d3.forceSimulation(nodes)
             .force("link", d3.forceLink(links).id(d => d.id))
             .force("charge", d3.forceManyBody().strength(-100))
             .force("center", d3.forceCenter(width / 2, height / 2));

         // Add links
         var link = svg.append("g")
             .attr("class", "links")
             .selectAll("line")
             .data(links)
             .enter().append("line")
             .attr("stroke", "#999")
             .attr("stroke-opacity", 0.6);

         // Add nodes
         
         var node = svg.append("g")
             .attr("class", "nodes")
             .selectAll("circle")
             .data(nodes)
             .enter().append("circle")
             .attr("r", 5)  // Radius of the nodes
             .attr("fill", "#69b3a2")
             .call(d3.drag()
                 .on("start", dragstarted)
                 .on("drag", dragged)
                 .on("end", dragended));

         // Add labels
         var label = svg.append("g")
             .attr("class", "labels")
             .selectAll("text")
             .data(nodes)
             .enter().append("text")
             .attr("dx", 12)
             .attr("dy", ".35em")
             .text(d => d.label || d.id);

         simulation.nodes(nodes).on("tick", ticked);
         simulation.force("link").links(links);

         function ticked() {{
             link.attr("x1", d => d.source.x)
                 .attr("y1", d => d.source.y)
                 .attr("x2", d => d.target.x)
                 .attr("y2", d => d.target.y);

             node.attr("cx", d => d.x)
                 .attr("cy", d => d.y);

             label.attr("x", d => d.x)
                  .attr("y", d => d.y);
         }}

         function dragstarted(d) {{
             if (!d3.event.active) simulation.alphaTarget(0.3).restart();
             d.fx = d.x;
             d.fy = d.y;
         }}

         function dragged(d) {{
             d.fx = d3.event.x;
             d.fy = d3.event.y;
         }}

         function dragended(d) {{
             if (!d3.event.active) simulation.alphaTarget(0);
             d.fx = null;
             d.fy = null;
         }}
         
      </script>
   </body>
</html>""")
